In [7]:
import csv
import numpy as np
import math
import matplotlib.pyplot as plt
import string
import scipy
import utils
import hrr_utils

from sklearn.decomposition import FastICA, PCA
from sklearn.preprocessing import normalize
from sklearn import manifold, datasets

from matplotlib import *
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import LineCollection
%matplotlib inline
"""
http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html
http://scikit-learn.org/stable/auto_examples/decomposition/plot_ica_vs_pca.html#sphx-glr-auto-examples-decomposition-plot-ica-vs-pca-py
"""
N = 1000
alphabet = string.ascii_lowercase + '#' + '.'
D = len(alphabet)
z = np.ones(N)

dic1 = hrr_utils.GenerateDefaultDictionary(N)
dic2 = hrr_utils.GenerateDefaultDictionary(N)

def genX(verbs):
    X = np.zeros((len(verbs), N)) # Exclusively difference PAST1-PRES1
    PRES1 = np.zeros((len(verbs), N))
    PRES2 = np.zeros((len(verbs), N))
    PAST1 = np.zeros((len(verbs), N))
    PAST2 = np.zeros((len(verbs), N))

    for m, pair in enumerate(verbs):
        past1 = hrr_utils.GetRVClip(pair[1], dic1, 3)
        past2 = hrr_utils.GetRVClip(pair[1], dic2, 3)
        pres1 = hrr_utils.GetRVClip(pair[0], dic1, 3)
        pres2 = hrr_utils.GetRVClip(pair[0], dic2, 3)
        PRES1[m] = pres1
        PRES2[m] = pres2
        PAST1[m] = past1
        PAST2[m] = past2

    #X = np.where(PAST1-PRES1 > 0, 1, -1)
    X = PAST1-PRES1
    return X, PRES1, PRES2, PAST1, PAST2


In [8]:
reg_pres, reg_past, reg_freq = utils.GetRegularVerbs(frequency=1)
irreg_pres, irreg_past, irreg_freq = utils.GetIrregularVerbs(frequency=1)


regular = zip(reg_pres, reg_past, reg_freq)
irregular = zip(irreg_pres, irreg_past, irreg_freq)

train_frac = 2./3.

train_reg = regular[0:int(train_frac*len(regular))]
train_irreg = irregular[0:int(train_frac*len(irregular))]


test_reg = regular[int(train_frac*len(regular)):]
test_irreg = irregular[int(train_frac*len(irregular)):]

dic1 = hrr_utils.GenerateDefaultDictionary(N)
dic2 = hrr_utils.GenerateDefaultDictionary(N)

regular.extend(irregular)
verbs = regular

In [9]:
trainX, trainpres1, trainpres2, trainpast1, trainpast2 = genX(train_reg + train_irreg)
testX, testpres1, testpres2, testpast1, testpast2 = genX(test_reg + test_irreg)

In [24]:
n_components = 2 
n_neighbors = 10
method = 'hessian'
testX = testX[:10]
print testX.shape
print testX

print np.isnan(testX).any()
print np.isinf(testX).any()

threshold_for_bug = 0.00000001 # could be any value, ex numpy.min
testX[testX < threshold_for_bug] = threshold_for_bug
print "compressing data"

fig = plt.figure(figsize=(15, 8))

tsne = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
Y = tsne.fit_transform(testX)

#plt.scatter(Y[:, 0], Y[:, 1], cmap=plt.cm.Spectral)
#plt.show()
"""
se = manifold.SpectralEmbedding(n_components=n_components,n_neighbors=n_neighbors)
Y = se.fit_transform(trainX)
plt.scatter(Y[:, 0], Y[:, 1], cmap=plt.cm.Spectral)
plt.show()
"""






(10, 1000)
[[  1.00000000e-08   1.00000000e-08   1.00000000e-08 ...,   1.00000000e-08
    1.00000000e-08   1.00000000e-08]
 [  1.00000000e-08   1.00000000e-08   1.00000000e-08 ...,   1.00000000e-08
    2.00000000e+00   2.00000000e+00]
 [  1.00000000e-08   1.00000000e-08   1.00000000e-08 ...,   1.00000000e-08
    1.00000000e-08   1.00000000e-08]
 ..., 
 [  2.00000000e+00   1.00000000e-08   2.00000000e+00 ...,   1.00000000e-08
    2.00000000e+00   2.00000000e+00]
 [  1.00000000e-08   1.00000000e-08   1.00000000e-08 ...,   1.00000000e-08
    1.00000000e-08   1.00000000e-08]
 [  1.00000000e-08   1.00000000e-08   1.00000000e-08 ...,   1.00000000e-08
    1.00000000e-08   1.00000000e-08]]
False
False
compressing data


ValueError: array must not contain infs or NaNs